Importing libraries and scripts

In [ ]:
%matplotlib inline
import nbimport
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import scipy.sparse.linalg as spla
import numpy.linalg as npl
from pyamg import smoothed_aggregation_solver, rootnode_solver
np.set_printoptions(linewidth=130)

In [2]:
from fsmfuncs import *
from ibmfuncs import *
from gridfuncs import *

importing Jupyter notebook from fsmfuncs.ipynb
importing Jupyter notebook from ibmfuncs.ipynb
importing Jupyter notebook from gridfuncs.ipynb


Numerical grid for the fluid

In [3]:
s1 = stretching(96+700, 0.04, 0.25, int(0.65/0.02)+700, 16, 16, 0.04)[0] # widening the uniform region around the cylinder by adding 700 points
s2 = stretching(96, 0.04, 0.25, int(0.65/0.02), 16, 16, 0.04)[0]
x = np.concatenate([-s2[::-1], s1[1:]])

s = stretching(96, 0.04, 0.25, int(0.65/0.04), 16, 16, 0.04)[0]
y = np.concatenate([-s[::-1], s[1:]])


n, m = len(y)-1, len(x)-1

dy, dx = np.diff(y), np.diff(x)
dxmin = min(np.min(dx), np.min(dy)) 

#Pressure
yp, xp = 0.5*(y[1:] + y[:-1]), 0.5*(x[1:] + x[:-1])                                                     
dyp, dxp = np.diff(yp), np.diff(xp)
p = np.zeros( (n, m) )

#Horizontal velocity
yu, xu = yp, x[1:-1]
u = np.zeros( (n, m-1) )

#Verticql velocity
yv, xv = y[1:-1], xp
v = np.zeros( (n-1, m) )

Immersed boundary

In [4]:
r_ = 0.5
l = int((2*np.pi*r_)/dxmin) 
ang_ = 2*np.pi*np.arange(l)/l 

#Coordinates
xi = 0 + r_*np.cos(ang_)
eta = 0 + r_*np.sin(ang_)

ds = 2*np.pi*r_/l*np.ones(l)

#Velocity: uniform horizontal translation 
uB = np.ones_like(xi)
vB = np.zeros_like(xi)

Plot grid and immersed boundary

In [ ]:
plt.figure(figsize=(8,8))
X, Y = np.meshgrid(x, y) 
plt.plot(X, Y, 'b-');
plt.plot(X.T, Y.T, 'b-'); 
plt.plot(xi, eta, 'ro-'); 
plt.axis('equal')

Boundary conditions + initial flow

In [6]:
uS, uN = np.zeros(m-1), np.zeros(m-1) 
uE, uW = np.zeros(n), np.zeros(n)

vS, vN = np.zeros(m), np.zeros(m)
vE, vW = np.zeros(n-1), np.zeros(n-1)

u[:,:]=0 
v[:,:]=0

Build matrices (I)

In [7]:
G, DuW, DuE, DvS, DvN = gradient(dxp, dyp)
R, iR = weight (dx, dy)
Mh, iMh = mass_hat (dxp, dyp)
Lh, Lux0, Lux1, Luy0, Luy1, Lvx0, Lvx1, Lvy0, Lvy1 = laplacian_hat(dx, dy, dxp, dyp)

L = Mh.dot(Lh.dot(iR))

M = Mh.dot(iR)
iM = R.dot(iMh)

iML = iM.dot(L)

In [ ]:
#Reynolds number
iRe = 1/40.0

#Time step
dt = 0.3* min(dxmin**2/iRe, dxmin) 

print(dt, dxmin**2/iRe, dxmin)

Build matrices (II)

In [9]:
A = (M/dt - 0.5*iRe*L).tocsr() 
B = (M/dt + 0.5*iRe*L).tocsr()

BN = dt*iM + (0.5*iRe)*dt**2*iML.dot(iM) + (0.5*iRe)**2*dt**3*iML.dot(iML.dot(iM))

In [10]:
#Velocity flux
q = R.dot(np.concatenate([u.ravel(), v.ravel()])) 
qast = q.copy()

Num1, Nvm1 = advection_hat(dx, dy, dxp, dyp, iR.dot(q),  uS, uN, uW, uE, vS, vN, vW, vE)
Nu, Nv = Num1, Nvm1

Time loop

In [ ]:
nt = int(3.5/dt)
print("Performing", nt, "steps")

# horizontal velocity of the cylinder
U0 = 1 

residuals = np.zeros(nt)

#Drag and lift coefficients
CFx = np.zeros(nt)
CFy = np.zeros(nt)

for k in range(nt):    
    
    # Computing regularization and interpolation operators
    Eh = interpolation_hat(xi, eta, ds, x, y, xp, yp, dx, dy, dxp, dyp)
    Hh = regularization_hat(xi, eta, ds, x, y, xp, yp, dx, dy, dxp, dyp)

    E = Eh.dot(iR)
    H = Mh.dot(Hh)


    Q = sp.hstack([G, E.T])

    QTBNQ = Q.T.dot(BN.dot(Q)).tocsr()
    
    ru = iRe*(Lux0.dot(uW) + Lux1.dot(uE) + Luy0.dot(uS) + Luy1.dot(uN)) - 1.5*Nu + 0.5*Num1
    rv = iRe*(Lvx0.dot(vW) + Lvx1.dot(vE) + Lvy0.dot(vS) + Lvy1.dot(vN)) - 1.5*Nv + 0.5*Nvm1
    
    bc1 = Mh.dot(np.concatenate([ru, rv]))
    r1 = B.dot(q.ravel()) + bc1
   
    # Solving the first linear equation using an iterative solver
    mls = smoothed_aggregation_solver(A)
    accelerated_residuals = []
    qast= mls.solve( r1, tol=1e-8, accel='cg', residuals=accelerated_residuals)
    
    bc2 = - (DuW.dot(uW*dy) + DuE.dot(uE*dy) + DvS.dot(vS*dx) + DvN.dot(vN*dx))
    r2 = np.concatenate([-bc2, uB, vB])

    # Solving the second linear equation using an iterative solver
    mls1 = smoothed_aggregation_solver(QTBNQ)
    accelerated_residuals = []
    λ = mls1.solve(Q.T.dot(qast)-r2, tol=1e-8, accel='cg', residuals=accelerated_residuals)
    
    # Pojection step
    qp1 = qast - BN.dot(Q.dot(λ))
    
    residuals[k] = la.norm(qp1-q)/(la.norm(dt*qp1)) 
    
    # Forcing term
    F = iMh.dot(E.T.dot(λ[n*m:]))
    Fx, Fy = F[:n*(m-1)].reshape((n, m-1)), F[n*(m-1):].reshape((n-1,m))
    CFx[k] = 2*np.sum(Fx*dxp[np.newaxis,:]*dy[:,np.newaxis]) 
    CFy[k] = 2*np.sum(Fy*dx[np.newaxis,:]*dyp[:,np.newaxis])  
    
    if k%100==0:
        print(k, k*dt, residuals[k], CFx[k], CFy[k])

    q = qp1
    uE = uE - dt/dx[-1]*(uE - iR.dot(q)[:n*(m-1)].reshape((n, m-1))[:,-1])
    
    # updating coordinates
    xi= xi + U0*dt 
    
    Num1, Nvm1 = Nu, Nv
    Nu, Nv = advection_hat(dx, dy, dxp, dyp, iR.dot(q), uS, uN, uW, uE, vS, vN, vW, vE)
    

Velocity, pressure, vorticity and forcing term

In [12]:
iRq = iR.dot(q)
u, v = iRq[:n*(m-1)].reshape((n, m-1)), iRq[n*(m-1):].reshape((n-1, m))
p = λ[:n*m].reshape((n,m))
f = λ[n*m:]
w = np.diff(v,axis=1)/dxp[np.newaxis,:]-np.diff(u,axis=0)/dyp[:,np.newaxis]

Display solution

In [ ]:
# Ploting velocity
x0, x1 = 0, 6
y0, y1 = -3, 3
plt.figure(figsize=(5.5*3,4))
plt.subplot(1,3,1) 
plt.pcolormesh(xu, yu, u, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

plt.subplot(1,3,2)
plt.pcolormesh(xv, yv, v, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

# Ploting pressure
plt.subplot(1,3,3)
plt.pcolormesh(xp, yp, p, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

In [ ]:
# Ploting vorticity
plt.figure(figsize=(5.5,4))
plt.pcolormesh(xu, yv, w, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

In [ ]:
# Ploting aerodynamic coefficients
plt.plot(np.arange(len(CFx))*dt, -CFx, 'b-', label='Cd')
plt.plot(np.arange(len(CFy))*dt, CFy, 'r:', label='Cl')
plt.xlim(0, 3.5)
plt.ylim(1, 10)
plt.xlabel('t')
plt.ylabel('Aerodynamic coefficients')
plt.legend()